<a href="https://colab.research.google.com/github/BlickandMorty/semantic-memory-system/blob/main/Memory_System_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chromadb sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46

In [3]:
import chromadb
from sentence_transformers import SentenceTransformer
import time

# 1. Initialize DB and Model
client = chromadb.Client()
collection = client.get_or_create_collection(name="ai_memory")
model = SentenceTransformer('all-MiniLM-L6-v2')

def add_memory(text, importance=1.0):
    """Stores a memory with a timestamp and importance weight."""
    embedding = model.encode(text).tolist()
    collection.add(
        embeddings=[embedding],
        documents=[text],
        metadatas=[{"importance": importance, "timestamp": time.time()}],
        ids=[str(time.time())]
    )

def retrieve_memory(query, n_results=3):
    """Retrieves memory. Logic: Similarity + Importance."""
    query_emb = model.encode(query).tolist()
    results = collection.query(query_embeddings=[query_emb], n_results=n_results)
    return results['documents'][0]

# --- TEST IT ---
add_memory("The user prefers concise Python code.", importance=5.0) # CRITICAL
add_memory("The weather in Jacksonville is 70 degrees.", importance=1.0) # TRIVIAL

print(f"Memory Check: {retrieve_memory('How should I write my code?')}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Memory Check: ['The user prefers concise Python code.', 'The weather in Jacksonville is 70 degrees.']


In [5]:
def retrieve_smart_memory(query, min_importance=3.0):
    """Only retrieves memories that are actually IMPORTANT."""
    query_emb = model.encode(query).tolist()

    # We add a 'where' filter to the database query
    results = collection.query(
        query_embeddings=[query_emb],
        n_results=3,
        where={"importance": {"$gte": min_importance}} # $gte means 'Greater Than or Equal to'
    )
    return results['documents'][0]

# --- TEST THE UPGRADE ---
print(f"Smart Search: {retrieve_smart_memory('Tell me about my preferences')}")

Smart Search: ['The user prefers concise Python code.']


In [6]:
def retrieve_time_aware_memory(query):
    """Retrieves memories but prints their age to show 'Time Awareness'."""
    query_emb = model.encode(query).tolist()
    results = collection.query(query_embeddings=[query_emb], n_results=2)

    for i, doc in enumerate(results['documents'][0]):
        timestamp = results['metadatas'][0][i]['timestamp']
        age_seconds = time.time() - timestamp
        print(f"Memory: {doc} | Age: {age_seconds:.2f} seconds ago")

# --- TEST IT ---
retrieve_time_aware_memory("What do I like?")

Memory: The user prefers concise Python code. | Age: 151.79 seconds ago
Memory: The weather in Jacksonville is 70 degrees. | Age: 151.76 seconds ago
